# SOTA INVESTIGATION FOR SENTIMENT ANALYSIS

## Load resources

In [ ]:
from emotion_classifier import *
import os

In [2]:
with open(os.path.join("secrets", "credentials.txt"), "r") as file:
    for line in file.readlines():
        key, value = line.split(":")

        if key == "huggingface":
            access_token = value

## Execute experiments

In [ ]:
available_datasets = list(DatasetHandler.DATASET_CONFIGS.keys())
print(f"The available datasets are: {available_datasets}")

In [ ]:
print("Available templates:")
PromptTemplate.TEMPLATES

### GO_EMOTIONS

In [4]:
dataset = DatasetHandler("go_emotions", "test")

RoBERTa Fine-tuned with go_emotions

In [ ]:
# Transformers model evaluation
model = TransformersStrategy("SamLowe/roberta-base-go_emotions")
classifier = EmotionClassifier(dataset, model)

classifier.evaluate()
_ = classifier.create_performance_report(print_report=True)

Phi 3.5 mini quantized - ZERO SHOT

In [7]:
system_prompt = f"""
You are an assistant that classifies text into ALL applicable emotions from: {", ".join(dataset.labels)}. 
"""
user_pre_prompt = f"""
From this text, select the predominant emotions and respond with the final comma-separated predominant emotion names only:
"""

In [ ]:
# Llama.cpp model evaluation
prompt_template = PromptTemplate( "llama", system_prompt, user_pre_prompt )
model = LlamaCppStrategy(
    model_name = "Phi-3.5-mini-instruct-Q4_K_M",
    model_path = "models/Phi-3.5-mini-instruct-Q4_K_M.gguf", 
    n_ctx = 4096,
    prompt_template = prompt_template
)
classifier = EmotionClassifier(dataset, model)
classifier.evaluate(verbose=True)
report = classifier.create_performance_report(print_report=True)

Phi 3.5 mini quantized - FEW SHOT

In [9]:
user_pre_prompt = f"""Classify sentiment based on these examples:

Text: It's wonderful because it's awful. At not with.
Emotions: admiration

Text: Lol! But I love your last name though. XD
Emotions: amusement, love

Text: Eff your video - love Canada 🇨🇦 Stupid geolock
Emotions: anger, annoyance

Text: [NAME] has towed the line of the Dark Side. He wouldn't cross it by doing something like this.
Emotions: annoyance, disapproval

Text: i mean, yeah tbh.
Emotions: approval, neutral

Text: please dont bring a woman in as a sex toy for you both. hire a sex worker instead
Emotions: caring

Text: Watch Vegan Gains’ video on that, he had it when he was like 13, highly doubt he was juicing then
Emotions: confusion, neutral

Text: It's great that you're a recovering addict, that's cool. Have you ever tried DMT?
Emotions: admiration, curiosity

Text: Translation. I wish I could afford it.
Emotions: desire

Text: Crap. I need more Excedrin. STAT.
Emotions: disappointment

Text: This sounds an awful lot like khorne......
Emotions: disgust

Text: I was teased for being a virgin when I was a 6th grader- in 2005
Emotions: embarrassment

Text: Kings fan here, good luck to you guys! Will be an interesting game to watch!
Emotions: excitement

Text: I've also heard that intriguing but also kinda scary
Emotions: fear

Text: I didn't know that, thank you for teaching me something today!
Emotions: gratitude

Text: [NAME] death is just so..... senseless. Why? WHY??? The based gods have forsaken us
Emotions: grief

Text: A surprise turn of events! I'm so glad you heard such great things about yourself!
Emotions: joy, surprise

Text: I got a lot of withholding to throw me into anxiety.
Emotions: nervousness

Text: I'm going to hold out hope for something minor even though it looked really bad. Just going to wait for the official news.
Emotions: optimism

Text: I only eat cronuts cuz I'm sophisticated!
Emotions: pride

Text: Some great advice right here!!!!!! I forgot about this
Emotions: admiration, realization

Text: Resetting a dislocated knee hurts like hell but it feels a lot better immediately after.
Emotions: relief

Text: I’m sorry to hear that friend :(. It’s for the best most likely if she didn’t accept you for who you are
Emotions: remorse

Text: I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!
Emotions: sadness

Text: They got bored from haunting earth for thousands of years and ultimately moved on to the afterlife.
Emotions: neutral

Text: 
"""

In [ ]:
# Llama.cpp model evaluation
prompt_template = PromptTemplate( "llama", system_prompt, user_pre_prompt)
model = LlamaCppStrategy(
    model_name = "Phi-3.5-mini-instruct-Q4_K_M-few_shot",
    model_path = "models/Phi-3.5-mini-instruct-Q4_K_M.gguf", 
    n_ctx = 4096,
    prompt_template = prompt_template
)
classifier = EmotionClassifier(dataset, model)
classifier.evaluate(verbose=True)
report = classifier.create_performance_report(print_report=True)

## Print results

In [ ]:
model_report = model.create_performance_report(print_report=True)

## Inference and model testing

In [ ]:
sentiment_template = f"Respond to this message only replying with the single emotion label you can detect from these emotion labels: {model.dataset_labels} for the following piece of text: "
model.run_inference(sentiment_template + "My mom said she wants me to go and never return back again")